In [ ]:
from google.colab import files
files.upload()

In [3]:
import gradio as gr
import pandas as pd
import numpy as np
import joblib

In [4]:
MODEL_PATH = "xgb_minimal_signature.pkl"
GENES_PATH = "minimal_signature_genes.pkl"
MEANS_PATH = "minimal_signature_means.pkl"

In [5]:
xgb_model = joblib.load(MODEL_PATH)
minimal_genes = list(joblib.load(GENES_PATH))

In [6]:
means_series = joblib.load(MEANS_PATH)
if not isinstance(means_series, pd.Series):
    means_series = pd.Series(means_series, index=minimal_genes, dtype=float)
means_series = means_series.reindex(minimal_genes).fillna(1.0)

In [7]:
try:
    model_feature_names = xgb_model.get_booster().feature_names
    if model_feature_names is None:
        model_feature_names = minimal_genes
except Exception:
    model_feature_names = minimal_genes

In [8]:
def predict_from_list(*vals):
    try:
        vals = [np.nan if v is None else float(v) for v in vals]
        input_df = pd.DataFrame([dict(zip(minimal_genes, vals))])
        input_df = input_df.reindex(columns=model_feature_names)
        input_df = input_df.fillna(means_series.reindex(model_feature_names))
        input_df = input_df.astype(float)

        prob = float(xgb_model.predict_proba(input_df)[:, 1][0])
        label = "HBV-induced HCC" if prob >= 0.5 else "Non-viral HCC"
        return prob, label
    except Exception as e:
        return 0.0, f"Error: {str(e)}"

In [9]:
inputs = [
    gr.Number(label=gene, value=float(means_series.get(gene, 1.0)), precision=4)
    for gene in minimal_genes
]

iface = gr.Interface(
    fn=predict_from_list,
    inputs=inputs,
    outputs=[
        gr.Number(label="Predicted Probability"),
        gr.Textbox(label="Prediction"),
    ],
    title="HBV vs Non-Viral HCC Classifier (Minimal Gene Signature)",
    description="Provide expression values for the minimal gene signature. Pre-filled with cohort means for convenience."
)

if __name__ == "__main__":
    iface.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://afc16d60864008f0ff.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
